In [52]:
import pandas as pd
import numpy as np


In [53]:
raw = pd.read_csv("Absenteeism_data.csv")
raw.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [54]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         700 non-null    int64  
 1   Reason for Absence         700 non-null    int64  
 2   Date                       700 non-null    object 
 3   Transportation Expense     700 non-null    int64  
 4   Distance to Work           700 non-null    int64  
 5   Age                        700 non-null    int64  
 6   Daily Work Load Average    700 non-null    float64
 7   Body Mass Index            700 non-null    int64  
 8   Education                  700 non-null    int64  
 9   Children                   700 non-null    int64  
 10  Pets                       700 non-null    int64  
 11  Absenteeism Time in Hours  700 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


In [55]:
raw["reason_group"] = pd.cut(
    x=raw["Reason for Absence"],
    bins=[-np.inf, 0, 14, 17, 21, np.inf],
    labels=["unknown", "sickness", "pregnancy", "accident", "other"],

)
 
reason_dummy = pd.get_dummies(raw["reason_group"], prefix="reason_group", drop_first=True)

In [56]:
df = pd.concat([raw, reason_dummy], axis=1)
#df.head(10)

In [57]:
df['timestamp'] = pd.to_datetime(df['Date'], dayfirst=True)
type(df['timestamp'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [58]:
df['month'] = df['timestamp'].dt.month
df['day_of_week'] = df['timestamp'].dt.day_of_week

In [59]:
df['higher_education'] = df['Education'].apply(lambda x: 1 if x > 1 else 0)


In [60]:
#target Einstufung
# der vorteil, der median zu benutyen ist, dass die Daten ausgleich geteilt werden 
benchmark = df['Absenteeism Time in Hours'].median()
df['target'] = df['Absenteeism Time in Hours'].apply(lambda x: 1 if x > benchmark else 0)
df.target.sum()/df.target.shape[0] # für logisticregression 40/60 aufgeteilt Daten funktioniert gut, aber für Neuralnetwork 50/50 ist besser

0.45571428571428574

In [61]:
df_select = df.copy()
#spalten die wir nicht brauchen
df_select.drop(columns=['Date', 'ID', 'Reason for Absence', 'Absenteeism Time in Hours', 'reason_group', 'timestamp', 'Education'], inplace=True)
#Rückwärtseliminierung
df_select.drop(columns=['month', 'Distance to Work', 'Daily Work Load Average'], inplace=True)

input = df_select.iloc[:, :-1]


In [62]:
dummy_column =['reason_group_sickness', 'reason_group_pregnancy',
       'reason_group_accident', 'reason_group_other', 'higher_education']
column_scaled = [x for x in input.columns if x not in dummy_column]


In [63]:
#die Daten satnardisieren
#dummy Spalten sollte  nicht skaliert werden, weil sie nicht interpretiert werden können
#normalerweise werden die Daten skaliert, vor dummy erstellung
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

input[column_scaled] = scaler.fit_transform(input[column_scaled])

In [64]:
#daten in train und test aufteilen und mischen
#standardmäßig schuffle = True
#stratify = True, damit die Verteilung der Zielvariable gleich bleibt
#stratofy link https://stackoverflow.com/questions/34842405/parameter-stratify-from-method-train-test-split-scikit-learn
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input, df_select['target'], test_size=0.2, random_state=42, stratify=df_select['target'], shuffle=True)

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


In [66]:
#Logistic Regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(x_train, y_train)
logreg.score(x_train, y_train)

0.7642857142857142

In [67]:
#überprüfung der Vorhersage
y_pred = logreg.predict(x_train)
np.sum(y_pred == y_train) / y_train.shape[0] 

0.7642857142857142

In [68]:
#erstellen der Confusion tabelle
feature_name = input.columns.values
summary = pd.DataFrame(data=feature_name, columns=["feature"])
summary['coefficient'] = logreg.coef_.reshape(-1)


In [69]:
#Achsenabschnitt hinzufügen
summary.loc[-1] = ['intercept', logreg.intercept_[0]]  # adding a row
summary.index = summary.index + 1  # shifting index
summary = summary.sort_index()  # sorting by index


In [70]:
#Odds Ratio hinzufügen
#wenn die Wahrscheinlichkeit 5/1 und die Odds Ratio 2 ist, für eine Einheitsänderung des Eingabe steigen die Wahrscheinlichkeit 2*5/1
summary['odds_ratio'] = np.exp(summary['coefficient'])

In [71]:
summary.sort_values(by='odds_ratio', ascending=False)

,feature,coefficient,odds_ratio
8,reason_group_accident,3.130692,22.889822
6,reason_group_sickness,2.765370,15.884919
9,reason_group_other,0.953105,2.593752
7,reason_group_pregnancy,0.763485,2.145741
1,Transportation Expense,0.597400,1.817387
4,Children,0.465434,1.592705
3,Body Mass Index,0.294789,1.342843
11,higher_education,0.202941,1.225001
2,Age,-0.154169,0.857127
10,day_of_week,-0.246102,0.781842


### interpretation
- Wenn jmd in reason_group_accident ist, ist er 22 mal wahrscheinlicher abwesend als jmd in reason_group_default.
- Wenn Transportation Expense um eine standardisierte Einheit steigt, ist die Absentismuswahrscheinlichkeit 1,78 mal so hoch wie im Basismodell.
- Wenn Age um eine standardisierte Einheit steigt, ist die Absentismuswahrscheinlichkeit 0,86 mal  geringer als im Basismodell.


wenn die Eingabe standarlisiert ist, ist es schwer, die Bedeutung zu interpretieren

wenn die Eingabe nicht standarlisiert ist, sinken die Genaurigkeit

am Besten bereitet beide Model vor

interpretieren logistic regression Koeffizienten https://stats.oarc.ucla.edu/other/mult-pkg/faq/general/faq-how-do-i-interpret-odds-ratios-in-logistic-regression/

### Backward Elimination
Rückwärtseliminierung ist eine Methode der Variablenauswahl, bei der man mit einem Modell startet, das alle Variablen enthält, und dann nacheinander die am wenigsten signifikanten Variablen entfernt. Dieses Verfahren wird oft in der Regression verwendet, um das Modell zu vereinfachen und zu verbessern. 
Normalerweise wird die Variable mit dem höchsten p-Wert (oder der geringsten Signifikanz) eliminiert, wenn der p-Wert über einem bestimmten Schwellenwert (z. B. 0,05) liegt. Aber hier fehlt uns der p-Wert, weil in ML sie glauben, dass ein ausreichend kleines Gewicht vernachlässigbar ist.

- month		1.079030
- Distance to Work		1.024061
- Daily Work Load Average 0.951270

hat odds_ratio fast gleich wie 1, deshalb entfernen wir diese Werte, und die Genaurigkeit hat nicht geändert

In [72]:
#model evaluieren
logreg.score(x_test, y_test)

0.7642857142857142

In [73]:
#die Wahrscheinlichkeit die Mitarbeiterin abwesent ist
y_pred_proba = logreg.predict_proba(x_test)[:, 1]


In [74]:
#Model speichern
import pickle
filename = 'model.sav'
pickle.dump(logreg, open(filename, 'wb'))
filename = 'scaler.sav'
pickle.dump(scaler, open(filename, 'wb'))

### Model Speichern Methoden
1. Joblib von Scipy
2. Json
3. pickle

potenzial Probleme mit pickle:

1. Benutzung von unterschiedliche Python Version für pickling und unpicking kann Problemen verursachen
2. pickle ist langsam für große Modell
3. pickle ist nicht sicher "Entpacken Sie niemals Daten, die Sie von einer nicht vertrauenswürdigen oder nicht authentifizierten Quelle erhalten haben."